# Packages

In [1]:
import os
                
import scipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from bioat.lib.libpandas import set_option
from bioat.lib.libpath import HOME
from pybedtools import BedTool
set_option()

INFO  @ 2023-08-18 23:33:38 bioat.lib.libpandas.set_option: set pandas: max_colwidth=40
INFO  @ 2023-08-18 23:33:38 bioat.lib.libpandas.set_option: set pandas: display.width=120
INFO  @ 2023-08-18 23:33:38 bioat.lib.libpandas.set_option: set pandas: display.max_columns=None
INFO  @ 2023-08-18 23:33:38 bioat.lib.libpandas.set_option: set pandas: display.max_rows=50


# classification for Unassorted off-target

## get df_arts_unassorted

In [2]:
df_arts_unassorted = pd.read_csv('../art_seqinfo/df_arts_unassorted.csv')
df_arts_unassorted

,chrom,start,end,region_index,align_chr_name,align_chr_start,align_chr_end,align_strand,align_dist_to_signal,align_N0_base,align_total_match,align_total_mismatch,align_degen_total_match,align_degen_total_mismatch,align_degen_num,align_total_gap,align_score,align_target_seq,align_info_state,align_query_seq,file_info,<sample>,TAS-Dep_stat,id
0,chr19,4182444,4182569,chr19_4182494_4182519,chr19,4182511,4182527,-,17,T,17,0,17,0,0,0,95.0,TACGCGGCGGGGCTGTC,|||||||||||||||||,TACGCGGCGGGGCTGTC,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,on-target: SIRT6,0
1,chr9,5021948,5022146,chr9_5021998_5022096,chr9,5022005,5022021,-,7,C,17,0,17,0,0,0,85.0,CCATTTCTGTCATCGTA,|||||||||||||||||,CCATTTCTGTCATCGTA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,on-target: JAK2,2
2,chr19,4182444,4182569,chr19_4182494_4182519,chr19,4182476,4182492,+,2,C,17,0,17,0,0,0,85.0,CCGGGAGGCCGCACTTG,|||||||||||||||||,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,on-target: SIRT6,4
3,chr9,5021875,5022047,chr9_5021925_5021997,chr9,5021972,5021988,+,9,C,17,0,17,0,0,0,85.0,CTGAAAAAGACTCTGCA,|||||||||||||||||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-2_T...,DetectSeq_JAK2-DddA11_REP-2,Left,5
4,chr9,5021948,5022146,chr9_5021998_5022096,chr9,5021972,5021988,+,10,C,17,0,17,0,0,0,85.0,CTGAAAAAGACTCTGCA,|||||||||||||||||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,on-target: JAK2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22942,chr12,119020881,119021002,chr12_119020931_119020952,chr12,119020986,119021001,-,34,A,8,8,8,8,0,0,8.0,AGAACACAGCGCCTCC,|XX|X|||XXXX||X|,ATTAAACACAAACTAC,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Left,27116
22943,chr14,72104166,72104275,chr14_72104216_72104225,chr14,72104242,72104258,-,17,G,6,11,9,8,3,0,7.0,GGAAAAGCAAGACAACG,XX.|||XX.|XX|X.|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27117
22944,chr2,236178158,236178271,chr2_236178208_236178221,chr2,236178168,236178184,+,24,T,7,10,7,10,0,0,5.0,TTCGCTCCCACTGGGCA,X|XXXXXXX|||XX|||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27125
22945,chr10,123135225,123135335,chr10_123135275_123135285,chr10,123135262,123135278,-,-13,G,8,9,8,9,0,0,4.0,GCGGACCAGGCCCTCCC,XX|X|XX||X|X||X|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27130


In [3]:
index_unassorted = df_arts_unassorted['region_index'].unique().tolist()
index_unassorted[:10]

['chr19_4182494_4182519',
 'chr9_5021998_5022096',
 'chr9_5021925_5021997',
 'chr11_2630829_2630903',
 'chr2_88149212_88149266',
 'chr9_69698553_69698603',
 'chr10_33333995_33334044',
 'chr13_52534773_52534853',
 'chrX_20442161_20442201',
 'chr13_51305640_51305728']

## check if ok than all ctcf binding sites have been removed

In [4]:
df_ctcf = pd.read_csv(f'{HOME}/1.database/public_data/CTCF_ATAC_data/ENCFF285QVL_CTCF_binding_sites_fix_range.bed',
                      sep='\t')


coor_ctcf_bed = BedTool.from_dataframe(df_ctcf.iloc[:, :3])
coor_ctcf_bed.to_dataframe()

coor_unassorted = pd.Series(index_unassorted).str.split('_', expand=True)
coor_unassorted.columns = ['chrom', 'start', 'end']
coor_unassorted[['start', 'end']] = coor_unassorted[['start', 'end']].astype(int)
coor_unassorted_bed = BedTool.from_dataframe(coor_unassorted)
df_coor_intersect = coor_unassorted_bed.intersect(coor_ctcf_bed, loj=True).to_dataframe()
df_coor_intersect.columns = ['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2']
df_coor_tas_indep = df_coor_intersect.query('start2 != -1').drop_duplicates(subset=['chrom1', 'start1', 'end1'])
df_coor_unassorted = df_coor_intersect.query('start2 == -1').drop_duplicates(subset=['chrom1', 'start1', 'end1'])
print(df_coor_tas_indep.empty)
df_coor_unassorted

True


,chrom1,start1,end1,chrom2,start2,end2
0,chr19,4182494,4182519,.,-1,-1
1,chr9,5021998,5022096,.,-1,-1
2,chr9,5021925,5021997,.,-1,-1
3,chr11,2630829,2630903,.,-1,-1
4,chr2,88149212,88149266,.,-1,-1
...,...,...,...,...,...,...
11703,chr2,22882093,22882127,.,-1,-1
11704,chr17,57697813,57697893,.,-1,-1
11705,chr1,10523435,10523475,.,-1,-1
11706,chr16,48609361,48609384,.,-1,-1


## from poission_res get highest site index

In [5]:
df_detect_seq_signal = pd.read_csv('../poisson_res/poisson_res_all.tsv.gz', sep='\t')

In [6]:
df_detect_seq_signal.columns

Index(['<sample>', 'chr_name', 'region_start', 'region_end', 'mpmat_index', 'region_site_num', 'region_block_site_num',
       'region_mut_site_num', 'region_site_index', 'region_block_state', 'region_highest_site_index',
       'region_highest_site_mut_num', 'region_highest_site_cover_num', 'region_highest_site_mut_ratio', 'ctrl_count',
       'treat_count', 'ctrl_mut_count', 'treat_mut_count', 'ctrl_count.norm', 'treat_count.norm',
       'ctrl_mut_count.norm', 'treat_mut_count.norm', 'count_info', 'log2_FC', 'log2_FC_mut', 'test_state', 'p_value',
       'FDR'],
      dtype='object')

In [7]:
df_detect_seq_signal_unassorted = df_detect_seq_signal[lambda df: df["mpmat_index"].map(lambda x: x in index_unassorted)][["mpmat_index", "treat_mut_count.norm", "<sample>", "region_highest_site_index"]].copy()

df_detect_seq_signal_unassorted

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index
3,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA
18,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA
21,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA
32,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA
42,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA
...,...,...,...,...
705621,chrX_154924102_154924198,0.041798,DetectSeq_SIRT6-DddA11_REP-2,chrX_154924110_CT
705636,chrX_155216206_155216224,0.106817,DetectSeq_SIRT6-DddA11_REP-2,chrX_155216224_GA
705639,chrX_155439898_155439954,0.027865,DetectSeq_SIRT6-DddA11_REP-2,chrX_155439946_GA
705641,chrX_155570076_155570156,0.000000,DetectSeq_SIRT6-DddA11_REP-2,chrX_155570155_GA


In [35]:
df_detect_seq_signal_unassorted

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index,chrom,chromStart,chromEnd
3,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA,chr1,268347,268747
18,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA,chr1,816430,816830
21,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA,chr1,968472,968872
32,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA,chr1,1692868,1693268
42,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA,chr1,1910480,1910880
...,...,...,...,...,...,...,...
705621,chrX_154924102_154924198,0.041798,DetectSeq_SIRT6-DddA11_REP-2,chrX_154924110_CT,chrX,154923910,154924310
705636,chrX_155216206_155216224,0.106817,DetectSeq_SIRT6-DddA11_REP-2,chrX_155216224_GA,chrX,155216024,155216424
705639,chrX_155439898_155439954,0.027865,DetectSeq_SIRT6-DddA11_REP-2,chrX_155439946_GA,chrX,155439746,155440146
705641,chrX_155570076_155570156,0.000000,DetectSeq_SIRT6-DddA11_REP-2,chrX_155570155_GA,chrX,155569955,155570355


In [36]:
df_for_motif = df_detect_seq_signal_unassorted.drop_duplicates(subset=['mpmat_index', '<sample>'])
df_for_motif
# 没有组间duplicate

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index,chrom,chromStart,chromEnd
3,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA,chr1,268347,268747
18,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA,chr1,816430,816830
21,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA,chr1,968472,968872
32,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA,chr1,1692868,1693268
42,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA,chr1,1910480,1910880
...,...,...,...,...,...,...,...
705621,chrX_154924102_154924198,0.041798,DetectSeq_SIRT6-DddA11_REP-2,chrX_154924110_CT,chrX,154923910,154924310
705636,chrX_155216206_155216224,0.106817,DetectSeq_SIRT6-DddA11_REP-2,chrX_155216224_GA,chrX,155216024,155216424
705639,chrX_155439898_155439954,0.027865,DetectSeq_SIRT6-DddA11_REP-2,chrX_155439946_GA,chrX,155439746,155440146
705641,chrX_155570076_155570156,0.000000,DetectSeq_SIRT6-DddA11_REP-2,chrX_155570155_GA,chrX,155569955,155570355


In [37]:
df_for_motif = df_for_motif.drop_duplicates(subset=['mpmat_index'])
df_for_motif

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index,chrom,chromStart,chromEnd
3,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA,chr1,268347,268747
18,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA,chr1,816430,816830
21,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA,chr1,968472,968872
32,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA,chr1,1692868,1693268
42,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA,chr1,1910480,1910880
...,...,...,...,...,...,...,...
100779,chrX_154924102_154924198,0.029804,DetectSeq_ATP8-DddA11_REP-1,chrX_154924125_CT,chrX,154923925,154924325
100794,chrX_155216206_155216224,0.000000,DetectSeq_ATP8-DddA11_REP-1,chrX_155216206_GA,chrX,155216006,155216406
100797,chrX_155439898_155439954,0.000000,DetectSeq_ATP8-DddA11_REP-1,chrX_155439915_GA,chrX,155439715,155440115
100799,chrX_155570076_155570156,0.059608,DetectSeq_ATP8-DddA11_REP-1,chrX_155570130_GA,chrX,155569930,155570330


In [12]:
df_for_motif = df_for_motif.drop_duplicates(subset=['region_highest_site_index'])
df_for_motif

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index
3,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA
18,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA
21,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA
32,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA
42,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA
...,...,...,...,...
100779,chrX_154924102_154924198,0.029804,DetectSeq_ATP8-DddA11_REP-1,chrX_154924125_CT
100794,chrX_155216206_155216224,0.000000,DetectSeq_ATP8-DddA11_REP-1,chrX_155216206_GA
100797,chrX_155439898_155439954,0.000000,DetectSeq_ATP8-DddA11_REP-1,chrX_155439915_GA
100799,chrX_155570076_155570156,0.059608,DetectSeq_ATP8-DddA11_REP-1,chrX_155570130_GA


In [13]:
df_for_motif['chrom'] = df_for_motif['region_highest_site_index'].str.split('_').str[0]
df_for_motif['chromStart'] = df_for_motif['region_highest_site_index'].str.split('_').str[1].astype(int) - 200
df_for_motif['chromEnd'] = df_for_motif['region_highest_site_index'].str.split('_').str[1].astype(int) + 200

In [14]:
df_for_motif

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index,chrom,chromStart,chromEnd
3,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA,chr1,268347,268747
18,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA,chr1,816430,816830
21,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA,chr1,968472,968872
32,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA,chr1,1692868,1693268
42,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA,chr1,1910480,1910880
...,...,...,...,...,...,...,...
100779,chrX_154924102_154924198,0.029804,DetectSeq_ATP8-DddA11_REP-1,chrX_154924125_CT,chrX,154923925,154924325
100794,chrX_155216206_155216224,0.000000,DetectSeq_ATP8-DddA11_REP-1,chrX_155216206_GA,chrX,155216006,155216406
100797,chrX_155439898_155439954,0.000000,DetectSeq_ATP8-DddA11_REP-1,chrX_155439915_GA,chrX,155439715,155440115
100799,chrX_155570076_155570156,0.059608,DetectSeq_ATP8-DddA11_REP-1,chrX_155570130_GA,chrX,155569930,155570330


In [15]:
df_coor_unassorted = df_for_motif[['chrom', "chromStart", "chromEnd"]].copy()
df_coor_unassorted

,chrom,chromStart,chromEnd
3,chr1,268347,268747
18,chr1,816430,816830
21,chr1,968472,968872
32,chr1,1692868,1693268
42,chr1,1910480,1910880
...,...,...,...
100779,chrX,154923925,154924325
100794,chrX,155216006,155216406
100797,chrX,155439715,155440115
100799,chrX,155569930,155570330


In [16]:
bed_unassorted = BedTool().from_dataframe(df_coor_unassorted)
print(bed_unassorted.head())
len(bed_unassorted)

chr1	268347	268747
 chr1	816430	816830
 chr1	968472	968872
 chr1	1692868	1693268
 chr1	1910480	1910880
 chr1	1942105	1942505
 chr1	1942117	1942517
 chr1	2214912	2215312
 chr1	2230469	2230869
 chr1	2363166	2363566
 None


11708

In [17]:
# 指定参考基因组序列信息，并从中获取每个序列信息
fasta_unassorted = bed_unassorted.getfasta(
    fi=f'{HOME}/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa')
fasta_unassorted

<BedTool(/var/folders/79/yd907p9x4537vqhgfrm0md680000gn/T/pybedtools.cs8vz24g.tmp)>

In [18]:
# 将region序列信息转为dataframe
df_seqinfo_unassorted = pd.DataFrame(fasta_unassorted.print_sequence().split('>')[1:], columns=['info'])
df_seqinfo_unassorted[['coordinate', 'sequence']] = df_seqinfo_unassorted['info'].str.strip().str.split('\n', expand=True)
df_seqinfo_unassorted

,info,coordinate,sequence
0,chr1:268347-268747\nggttcctaaataagat...,chr1:268347-268747,ggttcctaaataagatagctattacacaaagataaa...
1,chr1:816430-816830\ngggtgtggttacaagg...,chr1:816430-816830,gggtgtggttacaaggatgtttcctttataaAAATG...
2,chr1:968472-968872\nGGAGTCCCCTCCCCTC...,chr1:968472-968872,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3,chr1:1692868-1693268\nCACTGCAGGCCGAC...,chr1:1692868-1693268,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...
4,chr1:1910480-1910880\nttatacaagcagtt...,chr1:1910480-1910880,ttatacaagcagtttcagtcattttagtggccacct...
...,...,...,...
11703,chrX:154923925-154924325\ncacgccactg...,chrX:154923925-154924325,cacgccactgactccagcctgcgtgacagagcaaga...
11704,chrX:155216006-155216406\nGAAGCTCCCT...,chrX:155216006-155216406,GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...
11705,chrX:155439715-155440115\ngtgaaatgac...,chrX:155439715-155440115,gtgaaatgacaggaaacaaagcaaggaaagggaggg...
11706,chrX:155569930-155570330\ncgagcaaaat...,chrX:155569930-155570330,cgagcaaaataaccagctaacatcataatgacagga...


In [19]:
# 去掉序列中含有N的 region（可能坐落于端粒或中性粒，影响分析）
df_seqinfo_unassorted = df_seqinfo_unassorted.query('~sequence.str.upper().str.contains("N")').copy()
df_seqinfo_unassorted

,info,coordinate,sequence
0,chr1:268347-268747\nggttcctaaataagat...,chr1:268347-268747,ggttcctaaataagatagctattacacaaagataaa...
1,chr1:816430-816830\ngggtgtggttacaagg...,chr1:816430-816830,gggtgtggttacaaggatgtttcctttataaAAATG...
2,chr1:968472-968872\nGGAGTCCCCTCCCCTC...,chr1:968472-968872,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3,chr1:1692868-1693268\nCACTGCAGGCCGAC...,chr1:1692868-1693268,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...
4,chr1:1910480-1910880\nttatacaagcagtt...,chr1:1910480-1910880,ttatacaagcagtttcagtcattttagtggccacct...
...,...,...,...
11703,chrX:154923925-154924325\ncacgccactg...,chrX:154923925-154924325,cacgccactgactccagcctgcgtgacagagcaaga...
11704,chrX:155216006-155216406\nGAAGCTCCCT...,chrX:155216006-155216406,GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...
11705,chrX:155439715-155440115\ngtgaaatgac...,chrX:155439715-155440115,gtgaaatgacaggaaacaaagcaaggaaagggaggg...
11706,chrX:155569930-155570330\ncgagcaaaat...,chrX:155569930-155570330,cgagcaaaataaccagctaacatcataatgacagga...


In [20]:
# 整理dataframe
df_seqinfo_unassorted[['chrom', 'chromStart', 'chromEnd']] = (
    df_seqinfo_unassorted['coordinate']
    .str.replace(':', '\t')
    .str.replace('-', '\t')
    .str.split('\t', expand=True)
)
df_seqinfo_unassorted

,info,coordinate,sequence,chrom,chromStart,chromEnd
0,chr1:268347-268747\nggttcctaaataagat...,chr1:268347-268747,ggttcctaaataagatagctattacacaaagataaa...,chr1,268347,268747
1,chr1:816430-816830\ngggtgtggttacaagg...,chr1:816430-816830,gggtgtggttacaaggatgtttcctttataaAAATG...,chr1,816430,816830
2,chr1:968472-968872\nGGAGTCCCCTCCCCTC...,chr1:968472-968872,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...,chr1,968472,968872
3,chr1:1692868-1693268\nCACTGCAGGCCGAC...,chr1:1692868-1693268,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...,chr1,1692868,1693268
4,chr1:1910480-1910880\nttatacaagcagtt...,chr1:1910480-1910880,ttatacaagcagtttcagtcattttagtggccacct...,chr1,1910480,1910880
...,...,...,...,...,...,...
11703,chrX:154923925-154924325\ncacgccactg...,chrX:154923925-154924325,cacgccactgactccagcctgcgtgacagagcaaga...,chrX,154923925,154924325
11704,chrX:155216006-155216406\nGAAGCTCCCT...,chrX:155216006-155216406,GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...,chrX,155216006,155216406
11705,chrX:155439715-155440115\ngtgaaatgac...,chrX:155439715-155440115,gtgaaatgacaggaaacaaagcaaggaaagggaggg...,chrX,155439715,155440115
11706,chrX:155569930-155570330\ncgagcaaaat...,chrX:155569930-155570330,cgagcaaaataaccagctaacatcataatgacagga...,chrX,155569930,155570330


In [30]:
df_seqinfo_unassorted[['chromStart', 'chromEnd']] = df_seqinfo_unassorted[['chromStart', 'chromEnd']].astype(int)
df_seqinfo_unassorted = df_seqinfo_unassorted[['chrom', 'chromStart', 'chromEnd', 'sequence']].copy()
df_seqinfo_unassorted['sequence'] = df_seqinfo_unassorted['sequence'].str.upper()
df_seqinfo_unassorted

,chrom,chromStart,chromEnd,sequence
0,chr1,268347,268747,GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...
1,chr1,816430,816830,GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...
2,chr1,968472,968872,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3,chr1,1692868,1693268,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...
4,chr1,1910480,1910880,TTATACAAGCAGTTTCAGTCATTTTAGTGGCCACCT...
...,...,...,...,...
11703,chrX,154923925,154924325,CACGCCACTGACTCCAGCCTGCGTGACAGAGCAAGA...
11704,chrX,155216006,155216406,GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...
11705,chrX,155439715,155440115,GTGAAATGACAGGAAACAAAGCAAGGAAAGGGAGGG...
11706,chrX,155569930,155570330,CGAGCAAAATAACCAGCTAACATCATAATGACAGGA...


In [32]:
df_seqinfo_unassorted.isna().sum()

chrom         0
chromStart    0
chromEnd      0
sequence      0
dtype: int64

In [34]:
df_seqinfo_unassorted['idx'] = df_seqinfo_unassorted['chrom'] + '_' + (df_seqinfo_unassorted['chromStart'] + 200).astype(str)
df_seqinfo_unassorted

,chrom,chromStart,chromEnd,sequence,idx
0,chr1,268347,268747,GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...,chr1_268547
1,chr1,816430,816830,GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...,chr1_816630
2,chr1,968472,968872,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...,chr1_968672
3,chr1,1692868,1693268,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...,chr1_1693068
4,chr1,1910480,1910880,TTATACAAGCAGTTTCAGTCATTTTAGTGGCCACCT...,chr1_1910680
...,...,...,...,...,...
11703,chrX,154923925,154924325,CACGCCACTGACTCCAGCCTGCGTGACAGAGCAAGA...,chrX_154924125
11704,chrX,155216006,155216406,GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...,chrX_155216206
11705,chrX,155439715,155440115,GTGAAATGACAGGAAACAAAGCAAGGAAAGGGAGGG...,chrX_155439915
11706,chrX,155569930,155570330,CGAGCAAAATAACCAGCTAACATCATAATGACAGGA...,chrX_155570130


In [40]:
df_detect_seq_signal_unassorted['idx'] = df_detect_seq_signal_unassorted['region_highest_site_index'].str[:-3]
df_detect_seq_signal_unassorted

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index,chrom,chromStart,chromEnd,idx
3,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA,chr1,268347,268747,chr1_268547
18,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA,chr1,816430,816830,chr1_816630
21,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA,chr1,968472,968872,chr1_968672
32,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA,chr1,1692868,1693268,chr1_1693068
42,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA,chr1,1910480,1910880,chr1_1910680
...,...,...,...,...,...,...,...,...
705621,chrX_154924102_154924198,0.041798,DetectSeq_SIRT6-DddA11_REP-2,chrX_154924110_CT,chrX,154923910,154924310,chrX_154924110
705636,chrX_155216206_155216224,0.106817,DetectSeq_SIRT6-DddA11_REP-2,chrX_155216224_GA,chrX,155216024,155216424,chrX_155216224
705639,chrX_155439898_155439954,0.027865,DetectSeq_SIRT6-DddA11_REP-2,chrX_155439946_GA,chrX,155439746,155440146,chrX_155439946
705641,chrX_155570076_155570156,0.000000,DetectSeq_SIRT6-DddA11_REP-2,chrX_155570155_GA,chrX,155569955,155570355,chrX_155570155


In [41]:
df_for_motif_unassorted_add_seq = pd.merge(df_detect_seq_signal_unassorted, df_seqinfo_unassorted, on='idx', how='left')
df_for_motif_unassorted_add_seq

,mpmat_index,treat_mut_count.norm,<sample>,region_highest_site_index,chrom_x,chromStart_x,chromEnd_x,idx,chrom_y,chromStart_y,chromEnd_y,sequence
0,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_268547_GA,chr1,268347,268747,chr1_268547,chr1,268347.0,268747.0,GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...
1,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_816630_GA,chr1,816430,816830,chr1_816630,chr1,816430.0,816830.0,GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...
2,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_968672_GA,chr1,968472,968872,chr1_968672,chr1,968472.0,968872.0,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,chr1_1693068_GA,chr1,1692868,1693268,chr1_1693068,chr1,1692868.0,1693268.0,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...
4,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,chr1_1910680_GA,chr1,1910480,1910880,chr1_1910680,chr1,1910480.0,1910880.0,TTATACAAGCAGTTTCAGTCATTTTAGTGGCCACCT...
...,...,...,...,...,...,...,...,...,...,...,...,...
81951,chrX_154924102_154924198,0.041798,DetectSeq_SIRT6-DddA11_REP-2,chrX_154924110_CT,chrX,154923910,154924310,chrX_154924110,NaN,NaN,NaN,NaN
81952,chrX_155216206_155216224,0.106817,DetectSeq_SIRT6-DddA11_REP-2,chrX_155216224_GA,chrX,155216024,155216424,chrX_155216224,NaN,NaN,NaN,NaN
81953,chrX_155439898_155439954,0.027865,DetectSeq_SIRT6-DddA11_REP-2,chrX_155439946_GA,chrX,155439746,155440146,chrX_155439946,NaN,NaN,NaN,NaN
81954,chrX_155570076_155570156,0.000000,DetectSeq_SIRT6-DddA11_REP-2,chrX_155570155_GA,chrX,155569955,155570355,chrX_155570155,NaN,NaN,NaN,NaN


In [42]:
df_for_motif_unassorted_add_seq.isna().sum()

mpmat_index                      0
treat_mut_count.norm             0
<sample>                         0
region_highest_site_index        0
chrom_x                          0
chromStart_x                     0
chromEnd_x                       0
idx                              0
chrom_y                      47035
chromStart_y                 47035
chromEnd_y                   47035
sequence                     47035
dtype: int64

In [43]:
df_for_motif_unassorted_add_seq = df_for_motif_unassorted_add_seq[['mpmat_index', 'treat_mut_count.norm', '<sample>', 'sequence']].copy()
df_for_motif_unassorted_add_seq

,mpmat_index,treat_mut_count.norm,<sample>,sequence
0,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...
1,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...
2,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...
4,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,TTATACAAGCAGTTTCAGTCATTTTAGTGGCCACCT...
...,...,...,...,...
81951,chrX_154924102_154924198,0.041798,DetectSeq_SIRT6-DddA11_REP-2,NaN
81952,chrX_155216206_155216224,0.106817,DetectSeq_SIRT6-DddA11_REP-2,NaN
81953,chrX_155439898_155439954,0.027865,DetectSeq_SIRT6-DddA11_REP-2,NaN
81954,chrX_155570076_155570156,0.000000,DetectSeq_SIRT6-DddA11_REP-2,NaN


In [45]:
df_for_motif_unassorted_add_seq = df_for_motif_unassorted_add_seq.query('sequence.notna()')
df_for_motif_unassorted_add_seq

,mpmat_index,treat_mut_count.norm,<sample>,sequence
0,chr1_268547_268579,0.000000,DetectSeq_ATP8-DddA11_REP-1,GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...
1,chr1_816630_816666,0.000000,DetectSeq_ATP8-DddA11_REP-1,GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...
2,chr1_968672_968697,0.000000,DetectSeq_ATP8-DddA11_REP-1,GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3,chr1_1693068_1693084,0.005961,DetectSeq_ATP8-DddA11_REP-1,CACTGCAGGCCGACGGGGGTGGAAACACGCAGTTTT...
4,chr1_1910680_1910707,0.000000,DetectSeq_ATP8-DddA11_REP-1,TTATACAAGCAGTTTCAGTCATTTTAGTGGCCACCT...
...,...,...,...,...
81937,chrX_153496743_153496757,0.051086,DetectSeq_SIRT6-DddA11_REP-2,GTGGGAGCAGGTTGCGAGCCAGGGAGCAGAGAGGGC...
81942,chrX_154031921_154031945,0.274009,DetectSeq_SIRT6-DddA11_REP-2,GTGCCTGGGCCTGAGTCACCATCTAGAACATTCCCA...
81944,chrX_154313332_154313399,0.320451,DetectSeq_SIRT6-DddA11_REP-2,TCACATGTGTGTCTGAATTTTCTGATTGGTCAATTG...
81948,chrX_154457793_154457805,0.055731,DetectSeq_SIRT6-DddA11_REP-2,GAGGCTGAGGCGGGAGCATCGTTTGAGCCTGGGGCG...


In [52]:
df_for_motif_unassorted_add_seq = df_for_motif_unassorted_add_seq.sort_values('mpmat_index').drop_duplicates(subset='mpmat_index')[['mpmat_index', 'sequence']]
df_for_motif_unassorted_add_seq

,mpmat_index,sequence
30629,chr10_100175182_100175206,ACCAAAGAGGACTCAATTGGTAGATTTTAAATTGAT...
30630,chr10_100229657_100229680,GCCGCCGGTGCCCAGCCGCTCCCGCATCTCCCAGGG...
30631,chr10_100373544_100373555,GAATGGAGAGGAAGCGCCTCCCTCTCCACCCACTCT...
30632,chr10_100710753_100710783,GCGCTACACGACTCCGCTTCTGTTTTCCCCGTGTTG...
7217,chr10_100747051_100747100,TGGGGGTGGGGTGCCCCCTTCTGACCCTCATCTCTC...
...,...,...
11511,chrX_96089557_96089614,CAGGCTATCTAAAGTCAATGTGAAGGAAAGAATTCT...
23220,chrX_96177233_96177267,TCTTGGAAACAAGCTAAATCAATATGTCATGCAAAA...
23221,chrX_96252440_96252507,ATTTATCTTTGTAGAGACAGGGTCTCACTATGTTGG...
34930,chrX_97027605_97027643,CAACAGCCAAAAGATTGTTTGCAATAAAGACCTTAT...


In [57]:
df_arts_unassorted_add_seq = pd.merge(df_arts_unassorted, df_for_motif_unassorted_add_seq, left_on='region_index', right_on='mpmat_index', how='left')
df_arts_unassorted_add_seq

,chrom,start,end,region_index,align_chr_name,align_chr_start,align_chr_end,align_strand,align_dist_to_signal,align_N0_base,align_total_match,align_total_mismatch,align_degen_total_match,align_degen_total_mismatch,align_degen_num,align_total_gap,align_score,align_target_seq,align_info_state,align_query_seq,file_info,<sample>,TAS-Dep_stat,id,mpmat_index,sequence
0,chr19,4182444,4182569,chr19_4182494_4182519,chr19,4182511,4182527,-,17,T,17,0,17,0,0,0,95.0,TACGCGGCGGGGCTGTC,|||||||||||||||||,TACGCGGCGGGGCTGTC,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,on-target: SIRT6,0,chr19_4182494_4182519,GGCGTGTTGGCGTCCCCCGGCGCGCAGGGGCGTCAC...
1,chr9,5021948,5022146,chr9_5021998_5022096,chr9,5022005,5022021,-,7,C,17,0,17,0,0,0,85.0,CCATTTCTGTCATCGTA,|||||||||||||||||,CCATTTCTGTCATCGTA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,on-target: JAK2,2,chr9_5021998_5022096,CTTTTGTATGTTTTTGTAGAGATGAGGTTTCACCAT...
2,chr19,4182444,4182569,chr19_4182494_4182519,chr19,4182476,4182492,+,2,C,17,0,17,0,0,0,85.0,CCGGGAGGCCGCACTTG,|||||||||||||||||,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,on-target: SIRT6,4,chr19_4182494_4182519,GGCGTGTTGGCGTCCCCCGGCGCGCAGGGGCGTCAC...
3,chr9,5021875,5022047,chr9_5021925_5021997,chr9,5021972,5021988,+,9,C,17,0,17,0,0,0,85.0,CTGAAAAAGACTCTGCA,|||||||||||||||||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-2_T...,DetectSeq_JAK2-DddA11_REP-2,Left,5,chr9_5021925_5021997,CGCCCAGCTAATTTTTTTCTTTTGTATGTTTTTGTA...
4,chr9,5021948,5022146,chr9_5021998_5022096,chr9,5021972,5021988,+,10,C,17,0,17,0,0,0,85.0,CTGAAAAAGACTCTGCA,|||||||||||||||||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,on-target: JAK2,6,chr9_5021998_5022096,CTTTTGTATGTTTTTGTAGAGATGAGGTTTCACCAT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22942,chr12,119020881,119021002,chr12_119020931_119020952,chr12,119020986,119021001,-,34,A,8,8,8,8,0,0,8.0,AGAACACAGCGCCTCC,|XX|X|||XXXX||X|,ATTAAACACAAACTAC,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Left,27116,chr12_119020931_119020952,GGCTGGCGAGAGGTTTTATATCAAAATCACAGAAAG...
22943,chr14,72104166,72104275,chr14_72104216_72104225,chr14,72104242,72104258,-,17,G,6,11,9,8,3,0,7.0,GGAAAAGCAAGACAACG,XX.|||XX.|XX|X.|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27117,chr14_72104216_72104225,TACTGTATTCTCTCTAGGCGTTTCCCCCAGGAGCTT...
22944,chr2,236178158,236178271,chr2_236178208_236178221,chr2,236178168,236178184,+,24,T,7,10,7,10,0,0,5.0,TTCGCTCCCACTGGGCA,X|XXXXXXX|||XX|||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27125,chr2_236178208_236178221,GCTTGCCCCCACACTCCGTCTCCCCAGGGCCACCCC...
22945,chr10,123135225,123135335,chr10_123135275_123135285,chr10,123135262,123135278,-,-13,G,8,9,8,9,0,0,4.0,GCGGACCAGGCCCTCCC,XX|X|XX||X|X||X|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27130,chr10_123135275_123135285,AAAATACCGAGGTCGCCGGACCCCAAAGGGTAGGAA...


In [59]:
df_arts_unassorted_add_seq.drop(columns='mpmat_index', inplace=True)
df_arts_unassorted_add_seq

,chrom,start,end,region_index,align_chr_name,align_chr_start,align_chr_end,align_strand,align_dist_to_signal,align_N0_base,align_total_match,align_total_mismatch,align_degen_total_match,align_degen_total_mismatch,align_degen_num,align_total_gap,align_score,align_target_seq,align_info_state,align_query_seq,file_info,<sample>,TAS-Dep_stat,id,sequence
0,chr19,4182444,4182569,chr19_4182494_4182519,chr19,4182511,4182527,-,17,T,17,0,17,0,0,0,95.0,TACGCGGCGGGGCTGTC,|||||||||||||||||,TACGCGGCGGGGCTGTC,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,on-target: SIRT6,0,GGCGTGTTGGCGTCCCCCGGCGCGCAGGGGCGTCAC...
1,chr9,5021948,5022146,chr9_5021998_5022096,chr9,5022005,5022021,-,7,C,17,0,17,0,0,0,85.0,CCATTTCTGTCATCGTA,|||||||||||||||||,CCATTTCTGTCATCGTA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,on-target: JAK2,2,CTTTTGTATGTTTTTGTAGAGATGAGGTTTCACCAT...
2,chr19,4182444,4182569,chr19_4182494_4182519,chr19,4182476,4182492,+,2,C,17,0,17,0,0,0,85.0,CCGGGAGGCCGCACTTG,|||||||||||||||||,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,on-target: SIRT6,4,GGCGTGTTGGCGTCCCCCGGCGCGCAGGGGCGTCAC...
3,chr9,5021875,5022047,chr9_5021925_5021997,chr9,5021972,5021988,+,9,C,17,0,17,0,0,0,85.0,CTGAAAAAGACTCTGCA,|||||||||||||||||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-2_T...,DetectSeq_JAK2-DddA11_REP-2,Left,5,CGCCCAGCTAATTTTTTTCTTTTGTATGTTTTTGTA...
4,chr9,5021948,5022146,chr9_5021998_5022096,chr9,5021972,5021988,+,10,C,17,0,17,0,0,0,85.0,CTGAAAAAGACTCTGCA,|||||||||||||||||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,on-target: JAK2,6,CTTTTGTATGTTTTTGTAGAGATGAGGTTTCACCAT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22942,chr12,119020881,119021002,chr12_119020931_119020952,chr12,119020986,119021001,-,34,A,8,8,8,8,0,0,8.0,AGAACACAGCGCCTCC,|XX|X|||XXXX||X|,ATTAAACACAAACTAC,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Left,27116,GGCTGGCGAGAGGTTTTATATCAAAATCACAGAAAG...
22943,chr14,72104166,72104275,chr14_72104216_72104225,chr14,72104242,72104258,-,17,G,6,11,9,8,3,0,7.0,GGAAAAGCAAGACAACG,XX.|||XX.|XX|X.|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27117,TACTGTATTCTCTCTAGGCGTTTCCCCCAGGAGCTT...
22944,chr2,236178158,236178271,chr2_236178208_236178221,chr2,236178168,236178184,+,24,T,7,10,7,10,0,0,5.0,TTCGCTCCCACTGGGCA,X|XXXXXXX|||XX|||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27125,GCTTGCCCCCACACTCCGTCTCCCCAGGGCCACCCC...
22945,chr10,123135225,123135335,chr10_123135275_123135285,chr10,123135262,123135278,-,-13,G,8,9,8,9,0,0,4.0,GCGGACCAGGCCCTCCC,XX|X|XX||X|X||X|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27130,AAAATACCGAGGTCGCCGGACCCCAAAGGGTAGGAA...


In [60]:
df_arts_unassorted_add_seq.isna().sum()

chrom                         0
start                         0
end                           0
region_index                  0
align_chr_name                0
align_chr_start               0
align_chr_end                 0
align_strand                  0
align_dist_to_signal          0
align_N0_base                 0
align_total_match             0
align_total_mismatch          0
align_degen_total_match       0
align_degen_total_mismatch    0
align_degen_num               0
align_total_gap               0
align_score                   0
align_target_seq              0
align_info_state              0
align_query_seq               0
file_info                     0
<sample>                      0
TAS-Dep_stat                  0
id                            0
sequence                      0
dtype: int64

In [61]:
df_arts_unassorted_add_seq.groupby(['<sample>', 'TAS-Dep_stat'])['region_index'].count()

<sample>                      TAS-Dep_stat    
DetectSeq_ATP8-DddA11_REP-1   Left                 102
                              Right                 23
DetectSeq_ATP8-DddA6_REP-1    Left                 648
                              Right                213
DetectSeq_ATP8-DddAwt_REP-1   Left                  13
                              Right                  4
DetectSeq_JAK2-DddA11_REP-1   Left                 291
                              Right                276
                              on-target: JAK2        2
DetectSeq_JAK2-DddA11_REP-2   Left                 432
                              Right                289
DetectSeq_SIRT6-DddA11_REP-1  Left                2131
                              Right               2113
DetectSeq_SIRT6-DddA11_REP-2  Left                8159
                              Right               8249
                              on-target: SIRT6       2
Name: region_index, dtype: int64

In [62]:
os.makedirs('../art_seqinfo/unassorted', exist_ok=True)

In [65]:
df_arts_unassorted_add_seq = df_arts_unassorted_add_seq.query('~`TAS-Dep_stat`.str.contains("on-target")')
df_arts_unassorted_add_seq

,chrom,start,end,region_index,align_chr_name,align_chr_start,align_chr_end,align_strand,align_dist_to_signal,align_N0_base,align_total_match,align_total_mismatch,align_degen_total_match,align_degen_total_mismatch,align_degen_num,align_total_gap,align_score,align_target_seq,align_info_state,align_query_seq,file_info,<sample>,TAS-Dep_stat,id,sequence
3,chr9,5021875,5022047,chr9_5021925_5021997,chr9,5021972,5021988,+,9,C,17,0,17,0,0,0,85.0,CTGAAAAAGACTCTGCA,|||||||||||||||||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-2_T...,DetectSeq_JAK2-DddA11_REP-2,Left,5,CGCCCAGCTAATTTTTTTCTTTTGTATGTTTTTGTA...
5,chr11,2630779,2630953,chr11_2630829_2630903,chr11,2630835,2630851,-,6,T,12,5,17,0,5,0,85.0,TACACAGCAAAGCTGTC,|||.|.||...||||||,TACGCGGCGGGGCTGTC,../art/DetectSeq_SIRT6-DddA11_REP-1_...,DetectSeq_SIRT6-DddA11_REP-1,Left,7,GTATTCTTAAGTTGCCCGTATATTTATTTATCAGTG...
6,chr2,88149162,88149316,chr2_88149212_88149266,chr2,88149244,88149260,+,6,T,10,7,17,0,7,0,81.0,TACACAACAAAGCTATC,|||.|..|...|||.||,TACGCGGCGGGGCTGTC,../art/DetectSeq_SIRT6-DddA11_REP-1_...,DetectSeq_SIRT6-DddA11_REP-1,Left,11,ATAGATTCAATCCAAAAAGATCTTCCTTGAGGCACA...
7,chr9,69698503,69698653,chr9_69698553_69698603,chr9,69698570,69698586,-,17,T,12,5,16,1,4,0,78.0,TACCCAGCAAAGCTGTC,|||X|.||...||||||,TACGCGGCGGGGCTGTC,../art/DetectSeq_SIRT6-DddA11_REP-1_...,DetectSeq_SIRT6-DddA11_REP-1,Left,26,CCACCATTACAGTACTAGAGTATACTGATTTTGACA...
8,chr10,33333945,33334094,chr10_33333995_33334044,chr10,33334048,33334064,-,4,T,12,5,16,1,4,0,78.0,TACAAGACAGAGCTGTC,|||.X|.|.|.||||||,TACGCGGCGGGGCTGTC,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Left,27,TACTCCAGACAGATTACGGAAGGTGTACAAAGGCGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22942,chr12,119020881,119021002,chr12_119020931_119020952,chr12,119020986,119021001,-,34,A,8,8,8,8,0,0,8.0,AGAACACAGCGCCTCC,|XX|X|||XXXX||X|,ATTAAACACAAACTAC,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Left,27116,GGCTGGCGAGAGGTTTTATATCAAAATCACAGAAAG...
22943,chr14,72104166,72104275,chr14_72104216_72104225,chr14,72104242,72104258,-,17,G,6,11,9,8,3,0,7.0,GGAAAAGCAAGACAACG,XX.|||XX.|XX|X.|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27117,TACTGTATTCTCTCTAGGCGTTTCCCCCAGGAGCTT...
22944,chr2,236178158,236178271,chr2_236178208_236178221,chr2,236178168,236178184,+,24,T,7,10,7,10,0,0,5.0,TTCGCTCCCACTGGGCA,X|XXXXXXX|||XX|||,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27125,GCTTGCCCCCACACTCCGTCTCCCCAGGGCCACCCC...
22945,chr10,123135225,123135335,chr10_123135275_123135285,chr10,123135262,123135278,-,-13,G,8,9,8,9,0,0,4.0,GCGGACCAGGCCCTCCC,XX|X|XX||X|X||X|X,CTGAAAAAGACTCTGCA,../art/DetectSeq_JAK2-DddA11_REP-1_T...,DetectSeq_JAK2-DddA11_REP-1,Left,27130,AAAATACCGAGGTCGCCGGACCCCAAAGGGTAGGAA...


In [68]:
ls = []
for sample, df in df_arts_unassorted_add_seq.groupby('<sample>'):
    # 上面的一个cell中的代码已经证明了duplication是TAS-Dep_stat alignment过程引入的，简单去重即可
    df = df.drop_duplicates(subset=['region_index'], keep='first')
    ls.append(df)
df_arts_unassorted_add_seq = pd.concat(ls)

df_arts_unassorted_add_seq.to_csv('../art_seqinfo/df_arts_unassorted_add_seq.csv', index=False)

In [69]:
for sample, df in df_arts_unassorted_add_seq.groupby('<sample>'):
    # 上面的duplication都是0则说明duplication是TAS-Dep_stat alignment过程引入的，简单去重即可
    print(f'df.shape[0] = {df.shape[0]}')
    print('查看是否有duplication', df.duplicated(subset='region_index').sum())

print('=' * 20)
for g, df in df_arts_unassorted_add_seq.groupby(['<sample>', 'TAS-Dep_stat']):
    sample, dep_stat = g
    print(f'df.shape[0] = {df.shape[0]}')
    print('查看是否有duplication', df.duplicated(subset='region_index').sum())
    # 上面的duplication都是0则说明duplication是TAS-Dep_stat alignment过程引入的，简单去重即可


df.shape[0] = 109
查看是否有duplication 0
df.shape[0] = 664
查看是否有duplication 0
df.shape[0] = 15
查看是否有duplication 0
df.shape[0] = 319
查看是否有duplication 0
df.shape[0] = 473
查看是否有duplication 0
df.shape[0] = 2413
查看是否有duplication 0
df.shape[0] = 8577
查看是否有duplication 0
df.shape[0] = 87
查看是否有duplication 0
df.shape[0] = 22
查看是否有duplication 0
df.shape[0] = 480
查看是否有duplication 0
df.shape[0] = 184
查看是否有duplication 0
df.shape[0] = 11
查看是否有duplication 0
df.shape[0] = 4
查看是否有duplication 0
df.shape[0] = 191
查看是否有duplication 0
df.shape[0] = 128
查看是否有duplication 0
df.shape[0] = 240
查看是否有duplication 0
df.shape[0] = 233
查看是否有duplication 0
df.shape[0] = 1818
查看是否有duplication 0
df.shape[0] = 595
查看是否有duplication 0
df.shape[0] = 7162
查看是否有duplication 0
df.shape[0] = 1415
查看是否有duplication 0


In [71]:
df_arts_unassorted_add_seq

,chrom,start,end,region_index,align_chr_name,align_chr_start,align_chr_end,align_strand,align_dist_to_signal,align_N0_base,align_total_match,align_total_mismatch,align_degen_total_match,align_degen_total_mismatch,align_degen_num,align_total_gap,align_score,align_target_seq,align_info_state,align_query_seq,file_info,<sample>,TAS-Dep_stat,id,sequence
3669,chr7,123412912,123413069,chr7_123412962_123413019,chr7,123412967,123412978,-,5,A,7,5,12,0,5,0,50.0,ATAAACTTTAAT,||...||||..|,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,4235,GTCAATATTCCTTCACAAAGTAGAGATACTTGAAGA...
3776,chr19,45742345,45742506,chr19_45742395_45742456,chr19,45742447,45742458,+,-2,A,10,2,11,1,1,0,49.0,ATGAGCTTGGGT,|||.||||X|||,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,4373,AAACTCCTGGGCTCAAGGGACCATCCTGCCTTGGCC...
4817,chr11,113102877,113103036,chr11_113102927_113102986,chr11,113102970,113102981,+,5,A,9,3,11,1,2,0,47.0,ATGAAATTTGGT,|||..X||||||,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,5721,GACCATAGATTAGCAGTTCCATGGGAATGTGAAAGG...
5275,chr4,127523422,127523578,chr4_127523472_127523528,chr4,127523489,127523500,+,-28,T,7,5,10,2,3,0,46.0,TTGAAATTTGAT,X||..X||||.|,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,6221,GTTACATATCAAAATTCACACAGGTAAAAATTAAAA...
5328,chr22,19547402,19547545,chr22_19547452_19547495,chr22,19547450,19547461,-,-2,T,7,5,10,2,3,0,46.0,TTGAACTTTGAG,X||..|||||.X,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,6285,TCATCTTGTAAAACTGATACTCCATACCCGCCAAAC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22311,chr12,15347569,15347765,chr12_15347619_15347715,chr12,15347722,15347738,+,-23,T,8,9,9,8,1,0,21.0,TCTGTAGCCTTTACTAA,X|X|X||X|XXX|||X.,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26145,TAAAGAAAGAGACTGTCTTATATTCACTATTATCAT...
22360,chr4,150557335,150557499,chr4_150557385_150557449,chr4,150557428,150557444,+,5,A,6,11,11,6,5,0,21.0,ACAGGGAAGAACTGTTA,X|.||X..XX.|XX||.,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26206,GAAAGGCTTCCCACTGGCCCAATCTGGAAGAATTTG...
22412,chr8,10421053,10421176,chr8_10421103_10421126,chr8,10421063,10421079,+,24,C,9,8,10,7,1,0,20.0,CCGGGGATGTGAGCTGG,|||||X.XXX|XX||X|,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26273,GGAGAATCACTTGAGCCTGAGAGGTCCAAGCTGCAC...
22453,chr1,165880712,165880857,chr1_165880762_165880807,chr1,165880789,165880805,+,-43,C,9,8,10,7,1,0,20.0,CTTGGTGGTAGCTGTTA,|XX||X||XX||XX||.,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26345,GTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGATGTC...


In [74]:
def get_fasta(x):
    region_index = x['region_index']
    seq = x['sequence']
    return f'>region_index:{region_index}\n{seq}\n'


for sample, df in df_arts_unassorted_add_seq.groupby('<sample>'):
    # 上面的一个cell中的代码已经证明了duplication是TAS-Dep_stat alignment过程引入的，简单去重即可
    df = df.drop_duplicates(subset=['region_index'], keep='first')
    text = ''.join(df.apply(get_fasta, axis=1).tolist())
    with open(f'../art_seqinfo/unassorted/seqinfo_{sample}_unassorted.fa', 'wt') as f:
        f.write(text)
# wc -l *.fa
#      218 seqinfo_DetectSeq_ATP8-DddA11_REP-1_unassorted.fa
#     1328 seqinfo_DetectSeq_ATP8-DddA6_REP-1_unassorted.fa
#       30 seqinfo_DetectSeq_ATP8-DddAwt_REP-1_unassorted.fa
#      638 seqinfo_DetectSeq_JAK2-DddA11_REP-1_unassorted.fa
#      946 seqinfo_DetectSeq_JAK2-DddA11_REP-2_unassorted.fa
#     4826 seqinfo_DetectSeq_SIRT6-DddA11_REP-1_unassorted.fa
#    17154 seqinfo_DetectSeq_SIRT6-DddA11_REP-2_unassorted.fa
#    25140 total


# total=25140/2=12570

In [75]:
df_arts_unassorted_add_seq

,chrom,start,end,region_index,align_chr_name,align_chr_start,align_chr_end,align_strand,align_dist_to_signal,align_N0_base,align_total_match,align_total_mismatch,align_degen_total_match,align_degen_total_mismatch,align_degen_num,align_total_gap,align_score,align_target_seq,align_info_state,align_query_seq,file_info,<sample>,TAS-Dep_stat,id,sequence
3669,chr7,123412912,123413069,chr7_123412962_123413019,chr7,123412967,123412978,-,5,A,7,5,12,0,5,0,50.0,ATAAACTTTAAT,||...||||..|,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,4235,GTCAATATTCCTTCACAAAGTAGAGATACTTGAAGA...
3776,chr19,45742345,45742506,chr19_45742395_45742456,chr19,45742447,45742458,+,-2,A,10,2,11,1,1,0,49.0,ATGAGCTTGGGT,|||.||||X|||,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,4373,AAACTCCTGGGCTCAAGGGACCATCCTGCCTTGGCC...
4817,chr11,113102877,113103036,chr11_113102927_113102986,chr11,113102970,113102981,+,5,A,9,3,11,1,2,0,47.0,ATGAAATTTGGT,|||..X||||||,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,5721,GACCATAGATTAGCAGTTCCATGGGAATGTGAAAGG...
5275,chr4,127523422,127523578,chr4_127523472_127523528,chr4,127523489,127523500,+,-28,T,7,5,10,2,3,0,46.0,TTGAAATTTGAT,X||..X||||.|,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,6221,GTTACATATCAAAATTCACACAGGTAAAAATTAAAA...
5328,chr22,19547402,19547545,chr22_19547452_19547495,chr22,19547450,19547461,-,-2,T,7,5,10,2,3,0,46.0,TTGAACTTTGAG,X||..|||||.X,ATGGGCTTTGGT,../art/DetectSeq_ATP8-DddA11_REP-1_T...,DetectSeq_ATP8-DddA11_REP-1,Right,6285,TCATCTTGTAAAACTGATACTCCATACCCGCCAAAC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22311,chr12,15347569,15347765,chr12_15347619_15347715,chr12,15347722,15347738,+,-23,T,8,9,9,8,1,0,21.0,TCTGTAGCCTTTACTAA,X|X|X||X|XXX|||X.,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26145,TAAAGAAAGAGACTGTCTTATATTCACTATTATCAT...
22360,chr4,150557335,150557499,chr4_150557385_150557449,chr4,150557428,150557444,+,5,A,6,11,11,6,5,0,21.0,ACAGGGAAGAACTGTTA,X|.||X..XX.|XX||.,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26206,GAAAGGCTTCCCACTGGCCCAATCTGGAAGAATTTG...
22412,chr8,10421053,10421176,chr8_10421103_10421126,chr8,10421063,10421079,+,24,C,9,8,10,7,1,0,20.0,CCGGGGATGTGAGCTGG,|||||X.XXX|XX||X|,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26273,GGAGAATCACTTGAGCCTGAGAGGTCCAAGCTGCAC...
22453,chr1,165880712,165880857,chr1_165880762_165880807,chr1,165880789,165880805,+,-43,C,9,8,10,7,1,0,20.0,CTTGGTGGTAGCTGTTA,|XX||X||XX||XX||.,CCGGGAGGCCGCACTTG,../art/DetectSeq_SIRT6-DddA11_REP-2_...,DetectSeq_SIRT6-DddA11_REP-2,Right,26345,GTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGATGTC...


# Motif analysis
```bash
see file meme_chip_step_by_step.sh

DIR=/home/zhaohuanan/0.apps/meme_suite/meme-5.5.3
export PATH=$DIR/bin:$DIR/libexec/meme-5.5.3:$PATH

mkdir -p \
    motif_pre_processing \
    motif_discovery/meme \
    motif_discovery/streme \
    motif_scanning/fimo \
    motif_comparison/tomtom \
    motif_enrichment/centrimo \
    motif_enrichment/spamo


for i in `ls *.fa`
# for i in seqinfo_DetectSeq_ATP8-DddA11_REP-1_independent.fa  # for test
do
    ########################################
    #### files ####
    ########################################
    # motif_pre_processing
    SEQ_LENGTH=400
    METRICS=motif_pre_processing/${i}.metrics
    SEQ_CENTERED=motif_pre_processing/${i}.seq_centered
    BFILE=motif_pre_processing/${i}.background
    
    ### Motif Discovery
    ### 预测输入序列上的motif信息
    ### de novo motif discovery
    MEME=motif_discovery/meme/$i
    STREME=motif_discovery/streme/$i

    ### Motif Enrichment
    ### 分析已知的motif在输入序列上的富集情况
    CENTRIMO=motif_enrichment/centrimo/$i
    SPAMO=motif_enrichment/spamo/$i

    # ### Motif Scanning
    # ### 分析输入序列上可能的motif出现的位置
    FIMO=motif_scanning/fimo/$i
    
    # ### Motif Comparison
    # ### 比较不同motif之间的相似性
    TOMTOM=motif_comparison/tomtom/${i}
    

    
    echo '===================================================>'
    echo 'pre-processing' # √
    echo '===================================================>'
    getsize $i 1> $METRICS
    fasta-most -min 50 < $i 1>> $METRICS  # test
    fasta-center -dna -len $SEQ_LENGTH < $i 1> $SEQ_CENTERED
    fasta-get-markov -nostatus -nosummary -dna -m 2 $i $BFILE

 
    echo '===================================================>'
    echo 'Motif Discovery: de novo motif discovery' # √
    echo '===================================================>'
    echo -e '\trun MEME'
    meme -dna -nmotifs 3 -maxw 50 -bfile $BFILE -revcomp -oc $MEME $i
    # nmotifs: 最多报告3个motifs
    # revcomp: allow sites on + or - DNA strands
    # nostatus: do not print progress reports to terminal
    
    echo -e '\trun STREME'
    streme -dna --nmotifs 3 --maxw 30 --p $SEQ_CENTERED --oc $STREME
    # STREME: The maximum allowed motif width is 30


    echo '===================================================>'
    echo 'Motif Enrichment: 分析已知的motif在输入序列上的富集情况' # √
    echo '===================================================>'
    echo -e '\trun CentiMo'
    centrimo -seqlen $SEQ_LENGTH -bfile $BFILE $SEQ_CENTERED $MEME/meme.xml \
     $STREME/streme.xml \
     -oc $CENTRIMO
 
    echo -e '\trun SpaMo'
    spamo -bgfile $BFILE -keepprimary -oc $SPAMO $i $MEME/meme.xml $STREME/streme.xml

    
    echo '===================================================>'
    echo 'motif scanning'  # √
    echo '===================================================>'
    echo -e '\trun FIMO'
    fimo --oc $FIMO --bgfile $BFILE $MEME/meme.xml $i

    echo '===================================================>'
    echo 'motif comparison'  # √
    echo '===================================================>'

    echo -e '\trun TOMTOM'
    tomtom -oc $TOMTOM $MEME/meme.xml \
     $HOME/1.database/db_meme_suite/motif_databases/JASPAR/JASPAR2022_CORE_redundant_v2.meme


done
```

## run scripts above, wait and next
在veteran服务器上跑，剩下的继续在本地分析
多拆几个脚本出来

# STOP TODO 2023-08-19 00:54:44

homer findMotifsGenome using unassorted (TAS-independent) off-target bed information

In [ ]:
sort_keys = [
    'DetectSeq_SIRT6-DddA11_REP-1', 'DetectSeq_SIRT6-DddA11_REP-2',  # SIRT6
    'DetectSeq_JAK2-DddA11_REP-1', 'DetectSeq_JAK2-DddA11_REP-2',  # JAK2
    'DetectSeq_ATP8-DddA11_REP-1', 'DetectSeq_ATP8-DddA6_REP-1', 'DetectSeq_ATP8-DddAwt_REP-1',  # ATP8
]

# print(df_arts_tale_dep.head(2))
# print('-' * 100)
# print(df_arts.head(2))
df_arts

##### 

In [ ]:
ls = []
for sample in sort_keys:
    tas_dep_index = sorted(df_arts_tale_dep.query('`<sample>`==@sample')['region_index'].unique().tolist())
    tas_indep_index = sorted(df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')[
                                 'region_index'].unique().tolist())
    ls.append([sample, len(tas_dep_index), len(tas_indep_index), len(tas_dep_index) + len(tas_indep_index)])
    # |    | <sample>                     |   TAS-dep |   putative TAS-indep |   Total |
    # |---:|:-----------------------------|----------:|---------------------:|--------:|
    # |  0 | DetectSeq_SIRT6-DddA11_REP-1 |        68 |                 3331 |    3399 |
    # |  1 | DetectSeq_SIRT6-DddA11_REP-2 |       230 |                 9163 |    9393 |
    # |  2 | DetectSeq_JAK2-DddA11_REP-1  |        24 |                  797 |     821 |
    # |  3 | DetectSeq_JAK2-DddA11_REP-2  |       184 |                  573 |     757 |
    # |  4 | DetectSeq_ATP8-DddA11_REP-1  |       128 |                   87 |     215 |
    # |  5 | DetectSeq_ATP8-DddA6_REP-1   |       512 |                  260 |     772 |
    # |  6 | DetectSeq_ATP8-DddAwt_REP-1  |        18 |                   22 |      40 |

    df_out = df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')

    df_out[['chrom', 'start', 'end']] = df_out['region_index'].str.split('_', expand=True)
    df_out = df_out[['chrom', 'start', 'end', 'region_index', 'align_strand']].copy()
    df_out.insert(4, 'score', '.')
    df_out['start'] = df_out['start'].astype(int) - 30
    df_out['end'] = df_out['end'].astype(int) + 30
    df_out.to_csv(f'../art/from_art_putative_IND_{sample}.bed', header=False, index=False, sep='\t')
print(pd.DataFrame(ls, columns=['<sample>', 'TAS-dep', 'putative TAS-indep', 'Total']).to_markdown())

```bash
findMotifsGenome.pl from_art_putative_IND_DetectSeq_ATP6.1-DddSs.bed $HOME/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa homer_putative_IND_DetectSeq_ATP6.1-DddSs -size 50 -mask
...
...
...
```

## homer annotatePeaks for all off-target

```bash
cd final_list_after_igv_check

annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA6_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddAwt_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddAwt_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-2.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-2.homer_log &

bioat table merge -input_fmt tsv -output_fmt csv --input_header True --output_header True ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log homer.annoResults.merge.csv
```

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../annoResults/homer.annoResults.merge.csv')

filt = ['Promoter', '5UTR', '3UTR', 'Exon', 'Intron', 'TTS', 'Intergenic']

df = df[df['Total size (bp)'].str.isdecimal()].copy()
df.drop_duplicates(subset=['<sample>', 'Annotation'], keep='first', inplace=True)
df = df[df['Annotation'].map(lambda x: x in filt)].copy()
df.iloc[:, 2:] = df.iloc[:, 2:].astype(float)
df

In [ ]:
from lets_plot import *
from lets_plot.mapping import as_discrete

In [ ]:
LetsPlot.setup_html()

In [ ]:
(
        ggplot(data=df, mapping=aes(x=as_discrete("Annotation", order=-1), y="Log2 Ratio (obs/exp)", fill='<sample>'))
        + geom_bar(stat='identity', position='dodge', width=0.7)
        # + scale_x_discrete(labels=filt)
        + ggsize(1000, 500)
    # + facet_grid(y='<sample>', scales='free')
    # + scale_y_continuous(limits=(-2, 10))
)

### unassorted off-target
deeptools?

#### Final unassorted off-target

In [ ]:
df_arts_unassorted

# STOP TODO

homer findMotifsGenome using unassorted (TAS-independent) off-target bed information

In [ ]:
sort_keys = [
    'DetectSeq_SIRT6-DddA11_REP-1', 'DetectSeq_SIRT6-DddA11_REP-2',  # SIRT6
    'DetectSeq_JAK2-DddA11_REP-1', 'DetectSeq_JAK2-DddA11_REP-2',  # JAK2
    'DetectSeq_ATP8-DddA11_REP-1', 'DetectSeq_ATP8-DddA6_REP-1', 'DetectSeq_ATP8-DddAwt_REP-1',  # ATP8
]

# print(df_arts_tale_dep.head(2))
# print('-' * 100)
# print(df_arts.head(2))
df_arts

##### 

In [ ]:
ls = []
for sample in sort_keys:
    tas_dep_index = sorted(df_arts_tale_dep.query('`<sample>`==@sample')['region_index'].unique().tolist())
    tas_indep_index = sorted(df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')[
                                 'region_index'].unique().tolist())
    ls.append([sample, len(tas_dep_index), len(tas_indep_index), len(tas_dep_index) + len(tas_indep_index)])
    # |    | <sample>                     |   TAS-dep |   putative TAS-indep |   Total |
    # |---:|:-----------------------------|----------:|---------------------:|--------:|
    # |  0 | DetectSeq_SIRT6-DddA11_REP-1 |        68 |                 3331 |    3399 |
    # |  1 | DetectSeq_SIRT6-DddA11_REP-2 |       230 |                 9163 |    9393 |
    # |  2 | DetectSeq_JAK2-DddA11_REP-1  |        24 |                  797 |     821 |
    # |  3 | DetectSeq_JAK2-DddA11_REP-2  |       184 |                  573 |     757 |
    # |  4 | DetectSeq_ATP8-DddA11_REP-1  |       128 |                   87 |     215 |
    # |  5 | DetectSeq_ATP8-DddA6_REP-1   |       512 |                  260 |     772 |
    # |  6 | DetectSeq_ATP8-DddAwt_REP-1  |        18 |                   22 |      40 |

    df_out = df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')

    df_out[['chrom', 'start', 'end']] = df_out['region_index'].str.split('_', expand=True)
    df_out = df_out[['chrom', 'start', 'end', 'region_index', 'align_strand']].copy()
    df_out.insert(4, 'score', '.')
    df_out['start'] = df_out['start'].astype(int) - 30
    df_out['end'] = df_out['end'].astype(int) + 30
    df_out.to_csv(f'../art/from_art_putative_IND_{sample}.bed', header=False, index=False, sep='\t')
print(pd.DataFrame(ls, columns=['<sample>', 'TAS-dep', 'putative TAS-indep', 'Total']).to_markdown())

```bash
findMotifsGenome.pl from_art_putative_IND_DetectSeq_ATP6.1-DddSs.bed $HOME/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa homer_putative_IND_DetectSeq_ATP6.1-DddSs -size 50 -mask
...
...
...
```

## homer annotatePeaks for all off-target

```bash
cd final_list_after_igv_check

annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA6_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddAwt_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddAwt_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-2.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-2.homer_log &

bioat table merge -input_fmt tsv -output_fmt csv --input_header True --output_header True ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log homer.annoResults.merge.csv
```

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../annoResults/homer.annoResults.merge.csv')

filt = ['Promoter', '5UTR', '3UTR', 'Exon', 'Intron', 'TTS', 'Intergenic']

df = df[df['Total size (bp)'].str.isdecimal()].copy()
df.drop_duplicates(subset=['<sample>', 'Annotation'], keep='first', inplace=True)
df = df[df['Annotation'].map(lambda x: x in filt)].copy()
df.iloc[:, 2:] = df.iloc[:, 2:].astype(float)
df

In [ ]:
from lets_plot import *
from lets_plot.mapping import as_discrete

In [ ]:
LetsPlot.setup_html()

In [ ]:
(
        ggplot(data=df, mapping=aes(x=as_discrete("Annotation", order=-1), y="Log2 Ratio (obs/exp)", fill='<sample>'))
        + geom_bar(stat='identity', position='dodge', width=0.7)
        # + scale_x_discrete(labels=filt)
        + ggsize(1000, 500)
    # + facet_grid(y='<sample>', scales='free')
    # + scale_y_continuous(limits=(-2, 10))
)

# Deeptools